In [1]:
import scCube
from scCube import scCube
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

/home/qjy/anaconda3/envs/sccube/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = scCube()

In [3]:
epochs = 100000
slice_list = ['151507', '151508', '151509', '151510', 
              '151669', '151670', '151671', '151672', 
              '151673', '151674', '151675', '151676'] 
running_time = []

In [ ]:
for i in slice_list:
    print('Training model using slice: ' + str(i) + ' data...')
    sc_data = pd.read_csv('data/DLPFC/processed/' + str(i) + '_data.csv', index_col=0)
    sc_meta = pd.read_csv('data/DLPFC/processed/' + str(i) + '_meta.csv', index_col=0)
    
    sc_adata = model.pre_process(
        sc_data=sc_data, 
        sc_meta=sc_meta,
        is_normalized=False)
    
    model_name = 'slice' + str(i) + '_epoch100000'
    
    # train
    t1 = time.time()
    generate_sc_meta, generate_sc_data = model.train_vae_and_generate_cell(
        sc_adata=sc_adata,
        celltype_key='Cell_type',
        cell_key='Cell',
        target_num=None,
        batch_size=512,
        epoch_num=epochs,
        lr=0.0001,
        hidden_size=128,
        save_model=True,
        save_path='/home/qjy/workspace/scCube/save_model',
        project_name=model_name,
        used_device='cuda:0')
    t2 = time.time()
    
    running_time.append(t2 - t1)
    
    # generate
    t3 = time.time()
    generate_sc_data, generate_sc_meta = model.generate_pattern_reference(
        sc_adata=sc_adata,
        generate_sc_data=generate_sc_data,
        generate_sc_meta=generate_sc_meta,
        celltype_key='Cell_type',
        spatial_key=['x', 'y'],
        cost_metric='sqeuclidean'
    )
    t4 = time.time()
    running_time.append(t4 - t3)
    
    # saving
    generate_sc_data.to_csv('result/sccube_DLPFC_' + str(i) + '_epoch100000_data.csv')
    generate_sc_meta.to_csv('result/sccube_DLPFC_' + str(i) + '_epoch100000_meta.csv')

Training model using slice: 151507 data...
the input is count matrix, normalizing it firstly...
generating by the proportion of cell types of the input scRNA-seq data...
begin vae training...


Train Epoch: 99999: 100%|██████████| 100000/100000 [14:33:10<00:00,  1.91it/s, loss=0.4949, min_loss=0.4931] 


vae training done!
saving the trained vae model...
save trained vae in /home/qjy/workspace/scCube/save_model/slice151507_epoch100000.pth.


Generate Epoch: 0: 100%|██████████| 4221/4221 [00:00<00:00, 11367.63it/s]


generated done!
data have been prepared!
Training model using slice: 151508 data...
the input is count matrix, normalizing it firstly...
generating by the proportion of cell types of the input scRNA-seq data...
begin vae training...


Train Epoch: 57787:  58%|█████▊    | 57788/100000 [6:27:23<4:41:11,  2.50it/s, loss=0.5314, min_loss=0.5314] 

In [ ]:
slice_list_all = list(np.repeat(slice_list, 2))
mode = ['train', 'generate']*12

running_time_df = pd.DataFrame({'slice': slice_list_all, 'mode': mode, 'time':running_time})
running_time_df.to_csv('result/sccube_DLPFC_self_time.csv')

running_time_df